In [1]:
restartBool = True
import os
print(os.getcwd())

/Users/hannahzhang/Desktop/PIMCO 1B/PIMCO-Text2SQL/test


In [2]:
import sys
import os
import re
import time
import openai as OpenAI
from typing import List, Tuple, Dict
import json
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from rapidfuzz import fuzz
os.chdir("..")
from chatgpt_api import chat_prompt_revised
print(os.getcwd())

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


current_dir = os.path.dirname(os.path.abspath("/d/GithubRepos/PIMCO-Text2SQL"))
din_modules_path = os.path.join(current_dir, 'chatgpt_api')
sys.path.append(din_modules_path)

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
if not client.api_key:
    raise ValueError("OpenAI API key not configured")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hannahzhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hannahzhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahzhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/hannahzhang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/certifi/cacert.pem'


/Users/hannahzhang/Desktop/PIMCO 1B/PIMCO-Text2SQL


In [3]:
if restartBool:
    if os.path.isfile('temp_queries.json'):
        os.remove('temp_queries.json')
    if os.path.isfile('din_accuracy_120.csv'):
        os.remove('din_accuracy_120.csv')

In [4]:
curr = os.getcwd()
print(curr)
output_file = os.path.join(curr, 'all_outputs')
def append_to_file(output, qnum, filename=output_file):
    # Check if file exists
    output_filename= filename+str(qnum)+'.txt'
    if not os.path.exists(output_filename):
        with open(output_filename, 'w') as file:
            file.write("Test_Din Output Log\n")
            file.write("=" * 80 + "\n")
    # Append the output
    with open(output_filename, 'a') as file:
        file.write(output + "\n" + "=" * 80 + "\n")


/Users/hannahzhang/Desktop/PIMCO 1B/PIMCO-Text2SQL


In [5]:
SCHEMA_FILE = 'chatgpt_api/schema.json'
print(f"Expected schema path: {SCHEMA_FILE}")  


def format_schema_for_gpt(schema):
    if not schema:
        return "No schema available"
        
    formatted_schema = []
    tables = schema.get('schema', {}).get('tables', [])
    
    for table in tables:
        table_name = table.get('name')
        formatted_schema.append(f"\nTable: {table_name}")
        formatted_schema.append("Columns:")
        for column in table.get('columns', []):
            col_name = column.get('name')
            col_type = column.get('type')
            formatted_schema.append(f"- {col_name} ({col_type})")
    
    return "\n".join(formatted_schema)

def load_schema_from_json(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as f:
            schema = json.load(f)
        return schema
    except FileNotFoundError:
        raise FileNotFoundError(
            status_code=404,
            detail=f"Schema file not found at {file_path}"
        )
    except json.JSONDecodeError as e:
        raise Exception(
            status_code=500,
            detail=f"Error decoding JSON schema: {str(e)}"
        )



try:
    db_schema = load_schema_from_json(SCHEMA_FILE)
except Exception as e:
    db_schema = None

# schema_info = format_schema_for_gpt(db_schema)
# print(schema_info)
schema_info = chat_prompt_revised.schema_info


Expected schema path: chatgpt_api/schema.json


In [6]:
primary_keys = {
            'SUBMISSION': ['ACCESSION_NUMBER'],
            'REGISTRANT': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_INFO': ['ACCESSION_NUMBER'],
            'INTEREST_RATE_RISK': ['ACCESSION_NUMBER', 'INTEREST_RATE_RISK_ID'],
            'BORROWER': ['ACCESSION_NUMBER', 'BORROWER_ID'],
            'BORROW_AGGREGATE': ['ACCESSION_NUMBER', 'BORROW_AGGREGATE_ID'],
            'MONTHLY_TOTAL_RETURN': ['ACCESSION_NUMBER', 'MONTHLY_TOTAL_RETURN_ID'],
            'MONTHLY_RETURN_CAT_INSTRUMENT': ['ACCESSION_NUMBER', 'ASSET_CAT', 'INSTRUMENT_KIND'],
            'FUND_VAR_INFO': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_HOLDING': ['ACCESSION_NUMBER', 'HOLDING_ID'],
            'IDENTIFIERS': ['HOLDING_ID', 'IDENTIFIERS_ID'],
            'DEBT_SECURITY': [],  
            'DEBT_SECURITY_REF_INSTRUMENT': ['HOLDING_ID', 'DEBT_SECURITY_REF_ID'],
            'CONVERTIBLE_SECURITY_CURRENCY': ['HOLDING_ID', 'CONVERTIBLE_SECURITY_ID'],
            'REPURCHASE_AGREEMENT': ['HOLDING_ID'],
            'REPURCHASE_COUNTERPARTY': ['HOLDING_ID', 'REPURCHASE_COUNTERPARTY_ID'],
            'REPURCHASE_COLLATERAL': ['HOLDING_ID', 'REPURCHASE_COLLATERAL_ID'],
            'DERIVATIVE_COUNTERPARTY': ['HOLDING_ID', 'DERIVATIVE_COUNTERPARTY_ID'],
            'SWAPTION_OPTION_WARNT_DERIV': ['HOLDING_ID'],
            'DESC_REF_INDEX_BASKET': ['HOLDING_ID'],
            'DESC_REF_INDEX_COMPONENT': ['HOLDING_ID', 'DESC_REF_INDEX_COMPONENT_ID'],
            'DESC_REF_OTHER': ['HOLDING_ID', 'DESC_REF_OTHER_ID'],
            'FUT_FWD_NONFOREIGNCUR_CONTRACT': ['HOLDING_ID'],
            'FWD_FOREIGNCUR_CONTRACT_SWAP': ['HOLDING_ID'],
            'NONFOREIGN_EXCHANGE_SWAP': ['HOLDING_ID'],
            'FLOATING_RATE_RESET_TENOR': ['HOLDING_ID', 'RATE_RESET_TENOR_ID'],
            'OTHER_DERIV': ['HOLDING_ID'],
            'OTHER_DERIV_NOTIONAL_AMOUNT': ['HOLDING_ID', 'OTHER_DERIV_NOTIONAL_AMOUNT_ID'],
            'SECURITIES_LENDING': ['HOLDING_ID'],
            'EXPLANATORY_NOTE': ['ACCESSION_NUMBER', 'EXPLANATORY_NOTE_ID']
        }

foreign_keys = [
            # ACCESSION_NUMBER relationships
            'REGISTRANT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'INTEREST_RATE_RISK.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROWER.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROW_AGGREGATE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_TOTAL_RETURN.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_RETURN_CAT_INSTRUMENT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_VAR_INFO.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_REPORTED_HOLDING.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'EXPLANATORY_NOTE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'SUBMISSION.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',

            # HOLDING_ID relationships
            'IDENTIFIERS.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY_REF_INSTRUMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'CONVERTIBLE_SECURITY_CURRENCY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_AGREEMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COLLATERAL.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DERIVATIVE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SWAPTION_OPTION_WARNT_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_BASKET.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_COMPONENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_OTHER.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FUT_FWD_NONFOREIGNCUR_CONTRACT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FWD_FOREIGNCUR_CONTRACT_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'NONFOREIGN_EXCHANGE_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FLOATING_RATE_RESET_TENOR.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV_NOTIONAL_AMOUNT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SECURITIES_LENDING.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID'
        ]

In [7]:
# def explore_keys():
#     #"""Explore potential primary and foreign keys in the database"""
#     import sqlite3
    
#     # Connect to database
#     conn = sqlite3.connect('sqlite/nport.db')
#     cursor = conn.cursor()

#     # Get all tables
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#     tables = cursor.fetchall()

#     print("Database Key Analysis:")
#     print("-" * 80)

#     # Analyze each table
#     for table in tables:
#         table_name = table[0]
#         print(f"\nTable: {table_name}")

#         # Get column info
#         cursor.execute(f"PRAGMA table_info({table_name})")
#         columns = cursor.fetchall()
        
#         # Get sample count for potential key columns
#         for col in columns:
#             col_name = col[1]
#             # Check if column name contains potential key indicators
#             if any(key_term in col_name.lower() for key_term in ['_id', 'accession', 'number']):
#                 cursor.execute(f"""
#                     SELECT COUNT(*) total_rows, 
#                            COUNT(DISTINCT {col_name}) unique_values 
#                     FROM {table_name}
#                     WHERE {col_name} IS NOT NULL
#                 """)
#                 stats = cursor.fetchone()
#                 print(f"Column: {col_name}")
#                 print(f"Total rows: {stats[0]}")
#                 print(f"Unique values: {stats[1]}")
                
#                 # If unique values equals total rows, likely a key
#                 if stats[0] == stats[1] and stats[0] > 0:
#                     print(">>> Potential PRIMARY KEY <<<")

#         # Look for foreign key relationships
#         for col in columns:
#             col_name = col[1]
#             if col_name == 'ACCESSION_NUMBER':
#                 cursor.execute(f"""
#                     SELECT COUNT(*) FROM {table_name} t1
#                     WHERE EXISTS (
#                         SELECT 1 FROM FUND_REPORTED_INFO t2 
#                         WHERE t1.ACCESSION_NUMBER = t2.ACCESSION_NUMBER
#                     )
#                 """)
#                 match_count = cursor.fetchone()[0]
#                 if match_count > 0:
#                     print(f"Foreign Key: {table_name}.ACCESSION_NUMBER -> FUND_REPORTED_INFO.ACCESSION_NUMBER")
            
#             elif col_name == 'HOLDING_ID':
#                 cursor.execute(f"""
#                     SELECT COUNT(*) FROM {table_name} t1
#                     WHERE EXISTS (
#                         SELECT 1 FROM FUND_REPORTED_HOLDING t2 
#                         WHERE t1.HOLDING_ID = t2.HOLDING_ID
#                     )
#                 """)
#                 match_count = cursor.fetchone()[0]
#                 if match_count > 0:
#                     print(f"Foreign Key: {table_name}.HOLDING_ID -> FUND_REPORTED_HOLDING.HOLDING_ID")

#     conn.close()

# # Run the analysis
# explore_keys()

In [8]:
def GPT4_generation(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o", 
                messages=[{"role": "user", "content": prompt}],
                n = 1,
                stream = False,
                temperature=0.0,
                #max_tokens=600,
                top_p = 1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
                # Removed stop=["Q:"] as it cause issues
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(3)
            else:
                print("Max retries reached")
                return None
    return None

In [9]:
############################################ VALUE RETRIEVAL AND SCHEMA LINKING
def schema_linking_prompt_maker(question):
  instruction = "# Find the schema_links for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  fields = format_schema_for_gpt(db_schema)
  foreign_keys = "Foreign_keys = " + foreign_keys + '\n'
  prompt = instruction + chat_prompt_revised.schema_linking_prompt + fields +foreign_keys+ 'Q: "' + question + """"\nA: Let’s think step by step."""
  return prompt

class PSLsh:
    def __init__(self, vectors, n_planes=10, n_tables=5, seed: int = 42):
        self.n_planes = n_planes
        self.n_tables = n_tables
        self.hash_tables = [{} for _ in range(n_tables)]
        self.random_planes = []
        
        np.random.seed(seed)
        
        for _ in range(n_tables):
            planes = np.random.randn(vectors.shape[1], n_planes)
            self.random_planes.append(planes)
            
        self.num_vectors = vectors.shape[0]
        self.vectors = vectors
        self.build_hash_tables()

    def build_hash_tables(self):
        for idx in range(self.num_vectors):
            vector = self.vectors[idx].toarray()[0]
            hashes = self.hash_vector(vector)
            for i, h in enumerate(hashes):
                if h not in self.hash_tables[i]:
                    self.hash_tables[i][h] = []
                self.hash_tables[i][h].append(idx)

    def hash_vector(self, vector):
        hashes = []
        for planes in self.random_planes:
            projections = np.dot(vector, planes)
            hash_code = ''.join(['1' if x > 0 else '0' for x in projections])
            hashes.append(hash_code)
        return hashes

    def query(self, vector):
        hashes = self.hash_vector(vector)
        candidates = set()
        for i, h in enumerate(hashes):
            candidates.update(self.hash_tables[i].get(h, []))
        return candidates


class ValueRetrieval:
    financial_terms = {
            'total': ['total', 'sum', 'aggregate', 'combined'],
            'assets': ['asset', 'holdings', 'investments', 'securities'],
            'liabilities': ['liability', 'debt', 'obligations'],
            'net': ['net', 'pure', 'adjusted'],
            'fund': ['fund', 'portfolio', 'investment vehicle'],
            'return': ['return', 'yield', 'profit', 'gain'],
            'monthly': ['monthly', 'month', 'monthly basis'],
            'rate': ['rate', 'percentage', 'ratio'],
            'risk': ['risk', 'exposure', 'vulnerability']
        }
    
    def __init__(self, schema_path: str = 'chatgpt_api/schema.json', lsh_seed: int = 42):
        load_dotenv()
        self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        # Load schema
        print("DEBUG: Loading schema file:", schema_path)
        with open(schema_path, 'r') as f:
            self.schema = json.load(f)

        # Initialize lemmatizer and stop words
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Build column name index
        self.column_index = self._build_column_index()        

        # Build vectorizer and LSH for backup matching
        self.build_vectorizer_and_lsh(seed=lsh_seed)
        
        # Get schema relationships
        self.primary_keys, self.foreign_keys = self.discover_schema_relationships()

    def _build_column_index(self) -> Dict:
        column_index = {}
        tables = self.schema.get('schema', {}).get('tables', [])
        
        for table in tables:
            table_name = table.get('name', '').lower()
            for column in table.get('columns', []):
                column_name = column.get('name', '').lower()
                
                # Store the full qualified name and column properties
                qualified_name = f"{table_name}.{column_name}"
                column_index[qualified_name] = {
                    'table': table_name,
                    'column': column_name,
                    'type': column.get('type', ''),
                    'words': self._split_column_name(column_name),
                    'synonyms': self._get_column_synonyms(column_name)
                }
                
        return column_index

    def _split_column_name(self, column_name: str) -> List[str]:
        """Split column name into individual words."""
        # Handle  underscore + camel case.
        words = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', column_name)).split()
        words.extend(column_name.split('_'))
        return [word.lower() for word in words if word]

    def _get_column_synonyms(self, column_name: str) -> List[str]:
        """Get synonyms for words in column name."""
        words = self._split_column_name(column_name)
        synonyms = []
        
        for word in words:
            if word in self.financial_terms:
                synonyms.extend(self.financial_terms[word])
                
        return list(set(synonyms))

    def build_vectorizer_and_lsh(self, seed: int):
        self.vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), min_df=1, max_df=0.95)
        self.term_list = self.get_schema_terms()
        self.term_vectors = self.vectorizer.fit_transform(self.term_list)
        self.lsh = PSLsh(self.term_vectors, n_planes=10, n_tables=5)

    def get_schema_terms(self) -> List[str]:
        terms = []
        tables = self.schema.get('schema', {}).get('tables', [])
        for table in tables:
            table_name = table.get('name', '').lower()
            terms.append(table_name)
            for column in table.get('columns', []):
                column_name = column.get('name', '').lower()
                terms.append(f"{table_name}.{column_name}")
        return terms

    def discover_schema_relationships(self):
        # Define our primary keys and foreign keys here
        primary_keys = {
            'SUBMISSION': ['ACCESSION_NUMBER'],
            'REGISTRANT': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_INFO': ['ACCESSION_NUMBER'],
            'INTEREST_RATE_RISK': ['ACCESSION_NUMBER', 'INTEREST_RATE_RISK_ID'],
            'BORROWER': ['ACCESSION_NUMBER', 'BORROWER_ID'],
            'BORROW_AGGREGATE': ['ACCESSION_NUMBER', 'BORROW_AGGREGATE_ID'],
            'MONTHLY_TOTAL_RETURN': ['ACCESSION_NUMBER', 'MONTHLY_TOTAL_RETURN_ID'],
            'MONTHLY_RETURN_CAT_INSTRUMENT': ['ACCESSION_NUMBER', 'ASSET_CAT', 'INSTRUMENT_KIND'],
            'FUND_VAR_INFO': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_HOLDING': ['ACCESSION_NUMBER', 'HOLDING_ID'],
            'IDENTIFIERS': ['HOLDING_ID', 'IDENTIFIERS_ID'],
            'DEBT_SECURITY': [],  
            'DEBT_SECURITY_REF_INSTRUMENT': ['HOLDING_ID', 'DEBT_SECURITY_REF_ID'],
            'CONVERTIBLE_SECURITY_CURRENCY': ['HOLDING_ID', 'CONVERTIBLE_SECURITY_ID'],
            'REPURCHASE_AGREEMENT': ['HOLDING_ID'],
            'REPURCHASE_COUNTERPARTY': ['HOLDING_ID', 'REPURCHASE_COUNTERPARTY_ID'],
            'REPURCHASE_COLLATERAL': ['HOLDING_ID', 'REPURCHASE_COLLATERAL_ID'],
            'DERIVATIVE_COUNTERPARTY': ['HOLDING_ID', 'DERIVATIVE_COUNTERPARTY_ID'],
            'SWAPTION_OPTION_WARNT_DERIV': ['HOLDING_ID'],
            'DESC_REF_INDEX_BASKET': ['HOLDING_ID'],
            'DESC_REF_INDEX_COMPONENT': ['HOLDING_ID', 'DESC_REF_INDEX_COMPONENT_ID'],
            'DESC_REF_OTHER': ['HOLDING_ID', 'DESC_REF_OTHER_ID'],
            'FUT_FWD_NONFOREIGNCUR_CONTRACT': ['HOLDING_ID'],
            'FWD_FOREIGNCUR_CONTRACT_SWAP': ['HOLDING_ID'],
            'NONFOREIGN_EXCHANGE_SWAP': ['HOLDING_ID'],
            'FLOATING_RATE_RESET_TENOR': ['HOLDING_ID', 'RATE_RESET_TENOR_ID'],
            'OTHER_DERIV': ['HOLDING_ID'],
            'OTHER_DERIV_NOTIONAL_AMOUNT': ['HOLDING_ID', 'OTHER_DERIV_NOTIONAL_AMOUNT_ID'],
            'SECURITIES_LENDING': ['HOLDING_ID'],
            'EXPLANATORY_NOTE': ['ACCESSION_NUMBER', 'EXPLANATORY_NOTE_ID']
        }

        foreign_keys = [
            # ACCESSION_NUMBER relationships
            'REGISTRANT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'INTEREST_RATE_RISK.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROWER.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROW_AGGREGATE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_TOTAL_RETURN.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_RETURN_CAT_INSTRUMENT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_VAR_INFO.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_REPORTED_HOLDING.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'EXPLANATORY_NOTE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'SUBMISSION.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',

            # HOLDING_ID relationships
            'IDENTIFIERS.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY_REF_INSTRUMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'CONVERTIBLE_SECURITY_CURRENCY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_AGREEMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COLLATERAL.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DERIVATIVE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SWAPTION_OPTION_WARNT_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_BASKET.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_COMPONENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_OTHER.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FUT_FWD_NONFOREIGNCUR_CONTRACT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FWD_FOREIGNCUR_CONTRACT_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'NONFOREIGN_EXCHANGE_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FLOATING_RATE_RESET_TENOR.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV_NOTIONAL_AMOUNT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SECURITIES_LENDING.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID'
        ]

        formatted_pks = []
        for table, keys in primary_keys.items():
            for key in keys:
                formatted_pks.append(f"{table}.{key}")

        return formatted_pks, foreign_keys

    def find_similar_words(self, word: str) -> List[Tuple[str, float]]:
        #"""Better matching using multiple techniques - backup method with financial terms dictionary."""
        if not word:
            return []

        word = word.lower()
        #print(f"\nDEBUG: Finding matches for '{word}'")
        
        matches = []
        
        # 1. Direct matching with column names and their components
        for qualified_name, metadata in self.column_index.items():
            score = 0.0
            
            # Check exact matches in column words
            if word in metadata['words']:
                matches.append((qualified_name, 1.0))
                continue
                
            # Check synonyms
            if word in self.financial_terms.get(word, []):
                matches.append((qualified_name, 0.9))
                continue
            
            # Fuzzy match with column words
            for col_word in metadata['words']:
                ratio = fuzz.ratio(word, col_word) / 100.0
                if ratio > score:
                    score = ratio
            
            # Fuzzy match with synonyms
            for term, synonyms in self.financial_terms.items():
                if term in metadata['words']:
                    for synonym in synonyms:
                        ratio = fuzz.ratio(word, synonym) / 100.0
                        if ratio > score:
                            score = ratio * 0.9  # Slightly lower weight for synonym matches
            
            if score > 0.6:  # Only include if similarity is above 60%
                matches.append((qualified_name, score))

        # 2. LSH-based matching as backup
        if len(matches) < 5:  # If we have fewer than 5 matches, try LSH
            try:
                word_vector = self.vectorizer.transform([word]).toarray()[0]
                candidate_indices = self.lsh.query(word_vector)
                
                for idx in candidate_indices:
                    term = self.term_list[idx]
                    if not any(term == m[0] for m in matches):  # Avoid duplicates
                        candidate_vector = self.term_vectors[idx].toarray()[0]
                        dist = np.linalg.norm(word_vector - candidate_vector)
                        sim = 1 / (1 + dist)
                        if sim > 0.5:  # Only include if similarity is above 50%
                            matches.append((term, sim * 0.8))
            except Exception as e:
                print(f"LSH matching failed: {e}")

        # Remove duplicates keeping highest score and sort by score
        unique_matches = {}
        for term, score in matches:
            if term not in unique_matches or score > unique_matches[term]:
                unique_matches[term] = score
        
        matches = [(term, score) for term, score in unique_matches.items()]
        matches.sort(key=lambda x: x[1], reverse=True)
        
        # Print debug info
        print(f"Found {len(matches)} matches for '{word}':")
        for match, score in matches[:5]:
            print(f"  {match}: {score:.4f}")
        
        return matches[:5] if matches else [('fund_reported_info.total_assets', 0.6)] if word in ['total', 'asset', 'assets'] else []
    
    def extract_keywords(self, question: str) -> Dict:
        system_prompt = """Given a financial database schema: {schema_info}

        Primary Keys: {primary_keys}

        Foreign Keys: {foreign_keys}

        Extract from the question schema-aware components using the examples below."""

        few_shot_examples = """
        ```
        Example Question: "Show me all equity-focused funds"
        {
        "keywords": ["equity", "funds", "series"],
        "keyphrases": ["equity-focused funds"], 
        "table_matches": ["FUND_REPORTED_INFO"],
        "column_matches": ["SERIES_NAME", "TOTAL_ASSETS"],
        "primary_keys": ["FUND_REPORTED_INFO.ACCESSION_NUMBER"]
        }

        Example Question: "Show fund holdings over 1 billion in assets"
        {
        "keywords": ["holdings", "assets", "funds"],
        "numerical_values": ["1 billion"],
        "table_matches": ["FUND_REPORTED_INFO", "FUND_REPORTED_HOLDING"],
        "column_matches": ["TOTAL_ASSETS", "SERIES_NAME", "HOLDING_VALUE"],
        "required_joins": [
            "FUND_REPORTED_INFO to FUND_REPORTED_HOLDING via ACCESSION_NUMBER"
        ],
        "primary_keys": [
            "FUND_REPORTED_INFO.ACCESSION_NUMBER",
            "FUND_REPORTED_HOLDING.HOLDING_ID"
        ],
        "foreign_keys": [
            "FUND_REPORTED_HOLDING.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER"
        ]
        }
        ```
        """

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt.format(
                    schema_info=self.schema,
                    primary_keys=self.primary_keys,
                    foreign_keys=self.foreign_keys
                )},
                {"role": "user", "content": few_shot_examples + f"```\nQuestion: {question}\n```"}
            ],
            tools=[{
                "type": "function",
                "function": {
                    "name": "extract_components",
                    "description": "Extract components mapping to schema",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "keywords": {"type": "array", "items": {"type": "string"}},
                            "keyphrases": {"type": "array", "items": {"type": "string"}},
                            "table_matches": {"type": "array", "items": {"type": "string"}},
                            "column_matches": {"type": "array", "items": {"type": "string"}},
                            "required_joins": {"type": "array", "items": {"type": "string"}},
                            "primary_keys": {"type": "array", "items": {"type": "string"}},
                            "foreign_keys": {"type": "array", "items": {"type": "string"}},
                            "numerical_values": {"type": "array", "items": {"type": "string"}}
                        },
                        "required": ["keywords", "table_matches", "column_matches"]
                    }
                }
            }],
            tool_choice={"type": "function", "function": {"name": "extract_components"}}
        )

        function_call = response.choices[0].message.tool_calls[0].function
        return json.loads(function_call.arguments)

    def preprocess_text(self, text: str) -> List[str]:
        """Tokenize and lemmatize input text, removing stop words."""
        if not text:  # Add check for empty text
            return []
            
        try:
            tokens = nltk.word_tokenize(str(text).lower())
            filtered_tokens = [word for word in tokens if word not in self.stop_words and word.isalnum()]
            lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in filtered_tokens]
            return lemmatized_tokens
        except Exception as e:
            print(f"Error in preprocessing text '{text}': {str(e)}")
            return []  # Return empty list instead of None on error
       
       
    def process_schema(self, question: str) -> str:
        # Get all the processing results
        results = self.process_question(question)
        
        # Organize schema links by type
        table_columns = []
        relevant_primary_keys = []
        relevant_foreign_keys = []
        
        # 1. Get main table/column matches
        for word, matches in results['similar_matches'].items():
            if matches:
                # Only take the top match if score > 0.7
                top_match = matches[0]  # (match, score)
                if top_match[1] > 0.7:
                    # Handle numerical values
                    if word in results['extracted_info'].get('numerical_values', []):
                        if 'billion' in word.lower():
                            table_columns.append(f"{top_match[0]} > 1000000000")
                        elif 'million' in word.lower():
                            table_columns.append(f"{top_match[0]} > 1000000")
                        else:
                            table_columns.append(f"{top_match[0]} > {word}")
                    else:
                        table_columns.append(top_match[0])
        
        # 2. Get relevant tables
        tables_needed = set()
        for link in table_columns:
            if '.' in link:
                tables_needed.add(link.split('.')[0].upper())
        
        # 3. Add relevant primary keys
        for pk in results['schema_relationships']['primary_keys']:
            table = pk.split('.')[0]
            if table in tables_needed:
                relevant_primary_keys.append(pk)
        
        # 4. Add relevant foreign keys
        for fk in results['schema_relationships']['foreign_keys']:
            tables_in_fk = set(part.split('.')[0] for part in fk.split(' = '))
            if tables_in_fk.intersection(tables_needed):
                relevant_foreign_keys.append(fk)
        print("Attempting to generate schema_links")
        counterIndex = 0
        schema_links = None
        while schema_links is None and counterIndex<3:
            try:
                schema_links = GPT4_generation(schema_linking_prompt_maker(question))
            except:
                print("Error while generating schema_link")
                counterIndex+=1
        try:
            schema_links = schema_links.split("Schema_links: ")[1]
        except:
            print("Slicing error for the schema_linking module")
            schema_links = "[]"

        # Format output with sections
        schema_dict = {
            "table_columns": table_columns,
            "primary_keys": relevant_primary_keys,
            "foreign_keys": relevant_foreign_keys,
            "schema_links": schema_links
        }
        
        print("\nProcessed Schema Links:")
        print("Table Columns:", table_columns)
        print("Primary Keys:", relevant_primary_keys)
        print("Foreign Keys:", relevant_foreign_keys)
        
        return schema_dict


    def process_question(self, question: str) -> Dict:
        # Extract keywords using gpt
        extracted_info = self.extract_keywords(question)

        words = []
        for key in ['keywords', 'keyphrases', 'named_entities', 'numerical_values']:
            words.extend(extracted_info.get(key, []))

        # Preprocess the words (lemmatize, remove stop words)
        processed_words = []
        for word in words:
            processed_words.extend(self.preprocess_text(word))

        # Remove duplicates
        processed_words = list(set(processed_words))

        # Find similar columns for each word
        similar_matches = {}
        for word in processed_words:
            similar_matches[word] = self.find_similar_words(word)

        # Combine the results
        result = {
            "question": question,
            "extracted_info": extracted_info,
            "processed_words": processed_words,
            "similar_matches": similar_matches,
            "schema_relationships": {
                "primary_keys": self.primary_keys,
                "foreign_keys": self.foreign_keys
            }
        }
        return result
    
global_vr = ValueRetrieval(schema_path='chatgpt_api/schema.json')

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/hannahzhang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/certifi/cacert.pem'


DEBUG: Loading schema file: chatgpt_api/schema.json


In [10]:
############################################ CLASSIFICATION
classification_prompt = '''
```
Q: "Find the filing date and submission number of all reports filed for an NPORT-P submission."
schema_links: [submission.filing_date, submission.sub_type = "NPORT-P", submission.accession_number]
A: Let’s think step by step. The SQL query for the question "Find the filing date and submission number of all reports filed for an NPORT-P submission." needs these tables = [submission], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Q: "Get the names and CIK of registrants who are located in California."
schema_links: [registrant.registrant_name, registrant.cik, registrant.state = "US-CA"]
A: Let’s think step by step. The SQL query for the question "Get the names and CIK of registrants who are located in California." needs these tables = [registrant], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Q: "Find the names and CIK of registrants in California, but only for those whose total assets are above 100 million."
schema_links: [registrant.registrant_name, registrant.cik, registrant.state = "US-CA", fund_reported_info.total_assets > 100000000]
A: Let's analyze this. The query involves data from two tables: "registrant" for registrant details and "fund_reported_info" for total assets. Since we need to check if total assets exceed 100 million, a nested query is necessary to filter based on this condition. This is a nested query. So, the SQL query can be classified as "NESTED."
Label: "NESTED"
```
'''

def classification_prompt_maker(question, schema_dict):
   instruction = """```
TASK OVERVIEW

Given the database schema:
{schema_info}

Relevant Columns:
{table_columns}

Relevant Primary Keys:
{primary_keys}

Relevant Foreign Keys:
{foreign_keys}

Schema Links:
{schema_links}

- For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN
- if need nested queries: predict NESTED
- elif need JOIN and don't need nested queries: predict NON-NESTED
- elif don't need JOIN and don't need nested queries: predict EASY

Consider table relationships and what joins would be needed.
```
"""

   prompt = instruction.format(
       schema_info=schema_info,
       table_columns=schema_dict["table_columns"],
       primary_keys=schema_dict["primary_keys"],
       foreign_keys=schema_dict["foreign_keys"],
       schema_links=schema_dict["schema_links"]
   ) + classification_prompt + f'Q: "{question}"\nschema_links: {schema_dict["schema_links"]}\nA: Let\'s think step by step.'
       
   return prompt

def process_question_classification(question, schema_dict):
    def extract_classification(text):
        print(f"Trying to extract classification from: {text}")
        # Common patterns in GPT's response
        patterns = [
            "Label:", 
            "Classification:", 
            "The SQL query can be classified as",
            "can be classified as"
        ]
        
        text = text.upper()  # Normalize text
        # Direct class detection
        for class_type in ["EASY", "NON-NESTED", "NESTED"]:
            if class_type in text:
                return class_type

        # Try splitting with different patterns
        for pattern in patterns:
            if pattern.upper() in text:
                parts = text.split(pattern.upper())
                if len(parts) > 1:
                    # Get the last part and clean it
                    result = parts[1].strip().strip('"').strip("'")
                    # Extract first word as classification
                    classification = result.split()[0].strip()
                    if classification in ["EASY", "NON-NESTED", "NESTED"]:
                        return classification
                        
        return "NESTED"  # Default fallback

    classification = None
    attempts = 0
    while classification is None and attempts < 3:
        try:
            print("Attempting classification...")
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{
                    "role": "user", 
                    "content": classification_prompt_maker(question, schema_dict) #### ADD SCHEMA LINKS
                }],
                n=1,
                stream=False,
                temperature=0.0,
                #max_tokens=300,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
            )
            raw_response = response.choices[0].message.content
            print("Raw response:", raw_response)
            classification = extract_classification(raw_response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            time.sleep(3)
            attempts += 1
            if attempts == 3:
                raise e
    
    final_class = classification if classification else "NESTED"
    return f'"{final_class}"', raw_response

############################################ SQL GENERATION
easy_example = '''
```
Example with reasoning process:

Q: "Find the issuers with a balance greater than 1 million."

schema_links: [fund_reported_holding.balance]

SQL: SELECT DISTINCT issuer_name 
      FROM fund_reported_holding 
      WHERE balance > 1000000;
```
'''

medium_example = '''
```
Example with reasoning process:

Q: "Find the total upfront payments and receipts for swaps with fixed rate receipts."

schema_links: [nonforeign_exchange_swap.upfront_payment, nonforeign_exchange_swap.upfront_receipt, nonforeign_exchange_swap.fixed_rate_receipt]

A: Let’s think step by step. For creating the SQL for the given question, we need to filter the swaps that have fixed rate receipts. Then, sum up the upfront payments and receipts. First, create an intermediate representation, then use it to construct the SQL query.
Intermediate_representation: 
SELECT SUM(nonforeign_exchange_swap.upfront_payment) + SUM(nonforeign_exchange_swap.upfront_receipt) 
FROM nonforeign_exchange_swap 
WHERE nonforeign_exchange_swap.fixed_rate_receipt IS NOT NULL;

SQL: 
SELECT SUM(upfront_payment) + SUM(upfront_receipt) 
FROM nonforeign_exchange_swap 
WHERE fixed_rate_receipt IS NOT NULL;
```
'''

hard_example = '''
```
Example with reasoning process:

Q: "Find the borrowers with aggregate value greater than $1 million and whose interest rate change at 10-year maturity for a 100 basis point change is positive."

schema_links: [borrower.aggregate_value, borrower.name, interest_rate_risk.intrst_rate_change_10yr_dv100]

A: Let's think step by step. First, we need to filter borrowers with aggregate values greater than $1 million. Then, we need to check for interest rate changes at 10-year maturity where the change is positive. 
The SQL query for the sub-question "What are the borrowers with aggregate value greater than $1 million and positive interest rate change at 10-year maturity for 100 basis points?" is:

Intermediate_representation: 
SELECT borrower.name 
FROM borrower 
JOIN interest_rate_risk 
ON borrower.accession_number = interest_rate_risk.accession_number 
WHERE borrower.aggregate_value > 1000000 
AND interest_rate_risk.intrst_rate_change_10yr_dv100 > 0;

SQL: 
SELECT borrower.name 
FROM borrower 
JOIN interest_rate_risk 
ON borrower.accession_number = interest_rate_risk.accession_number 
WHERE borrower.aggregate_value > 1000000 
AND interest_rate_risk.intrst_rate_change_10yr_dv100 > 0;
```
'''

def hard_prompt_maker(question, schema_dict, sub_questions=""):
   instruction = f"""```
OVERALL TASK:
I will provide a database schema, generate an SQL query that retrieves from the database the answer to this question: {question}
You might need join statements and nested queries for this.
```
"""+"""```
Relevant Columns:
{table_columns}

Relevant Primary Keys:
{primary_keys}

Relevant Foreign Keys:
{foreign_keys}

Schema Links:
{schema_links}
```
""".format(
       table_columns=schema_dict["table_columns"],
       primary_keys=schema_dict["primary_keys"],
       foreign_keys=schema_dict["foreign_keys"],
       schema_links=schema_dict["schema_links"]
   )+chat_prompt_revised.common_part_prompt
   
   #if sub_questions=="":
       #stepping = "A: Let's think step by step." # {question} can be solved by first solving a sub-question using nested queries.
   #else:
       #stepping = "A: Let's think step by step."# {question} can be solved by first solving the answer to the following sub-question {sub_questions}.

   prompt = f"""{instruction}{hard_example}
```
Q: "{question}"

schema_links: {schema_dict["schema_links"]}
```

A: Let's think step by step."""
   return prompt

def medium_prompt_maker(question, schema_dict):
   instruction = f"""```
OVERALL TASK:
I will provide a database schema, generate an SQL query that retrieves from the database the answer to this question: {question}
You should not need any nested queries, but you might need join statements for this question.
```
"""+"""```
Relevant Columns:
{table_columns}

Relevant Primary Keys:
{primary_keys}

Relevant Foreign Keys:
{foreign_keys}

Schema Links:
{schema_links}
```
""".format(
       table_columns=schema_dict["table_columns"],
       primary_keys=schema_dict["primary_keys"],
       foreign_keys=schema_dict["foreign_keys"],
       schema_links=schema_dict["schema_links"]
   )+chat_prompt_revised.common_part_prompt

   prompt = f"""{instruction}{medium_example}
```
Q: "{question}"

schema_links: {schema_dict["schema_links"]}
```

A: Let's think step by step."""
   return prompt

def easy_prompt_maker(question, schema_dict):
   instruction = f"""```
OVERALL TASK:
I will provide a database schema, generate an SQL query that retrieves from the database the answer to this question: {question}
You should not need any nested queries or join statements for this.
```
"""+"""```
Relevant Columns:
{table_columns}

Relevant Primary Keys:
{primary_keys}

Relevant Foreign Keys:
{foreign_keys}

Schema Links:
{schema_links}
```
""".format(
       table_columns=schema_dict["table_columns"],
       primary_keys=schema_dict["primary_keys"],
       foreign_keys=schema_dict["foreign_keys"],
       schema_links=schema_dict["schema_links"]
   )+chat_prompt_revised.common_part_prompt

   prompt = f"""{instruction}{easy_example} 
```
Q: "{question}"

schema_links: {schema_dict["schema_links"]}
```

SQL: """ #### ADD SCHEMA LINKS
   return prompt

In [11]:
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
import time
import json
from typing import Literal

thought_instructions = f"""
```
Thought Instructions:
```

```
Generate thoughts of increasing complexity.
Each thought should build on the previous ones and thoughts 
should progressively cover the nuances of the problem at hand.
```

```
First set of thoughts should be on whether a the query requires 
Common Table Expressions (CTEs) to calculate the
results for sub queries. 

Prefer using Common Table Expressions rather than
case when statements or nested subqueries.

If CTEs are required then for each CTE, an analysis of the purpose of each
CTE should be done.
An overall structure should be outlined as to what will be calculated in 
each CTE.
```

```
Next set of thoughts should on 
extracting out the names of as many of 
the relevant columns as possible for all CTEs and for all the sql clauses such as the 
`select`, `where` and `group_by` clauses.
There might be additions or deletions from this list based on the 
following additional thoughts to be generated.
```


```
Generate a thought to figure out the possible phrases in the query 
which can be used as values of the columns present in the table so as to use them 
in the `where` clause.
```

```
Generate a thought to compare these extracted values with the list of possible values
of columns listed in the information for the columns so as to use the exact string
in the `where` clause.
```

```
Generate a thought to reason whether `IS_TOP_TIER_ENTITY` flag is required or not.
```

```
Generate a thought to figure out which time period is being queried.
If nothing is specified use `PERIOD_ID = 2023Y`.
```

```
Generate a thought to figure out if a group_by clause is required.
```

```
The above thoughts about 
1. phrases for values of columns
2. query phrase to column value mapping
3. filters such as `ASSET_CAT` and others in the where clause
4. Period_id value to use
5. Group by column

should be generated for each of the CTE separately.
```

```
If the input question is similar to any of the examples given above,
then a thought should be generated to detect that and then that example 
should be followed closely to get the SQL for the input question given.
```

```
Closing Thoughts and Observations
```
These should summarize:
1. The structure of the SQL query:
    - This states whether the query has any nested query.
    If so, the structure of the nested query is also mentioned.
    If not, a summary of the function of each of the select`, `where`, `group_by` etc. clauses
    should be mentioned.
2. An explanation of how the query solves the user question.
"""

reasoning_instructions = """
```
1. Reasoning you provide should first focus on why a nested query was chosen or why it wasn't chosen.
2. It should give a query plan on how to solve this question - explain 
the mapping of the columns to the words in the input question.
3. It should explain each of the clauses and why they are structured the way they are structured. 
For example, if there is a `group_by`, an explanation should be given as to why it exists.
4. If there's any sum() or any other function used it should be explained as to why it was required.
```

```
Format the generated sql with proper indentation - the columns in the
(`select` statement should have more indentation than keyword `select`
and so on for each SQL clause.)
```
"""

# Final Output Schema
final_output_schema_json = json.dumps({
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "user_nlp_query": {
            "type": "string",
            "description": "The original natural language query to be translated into SQL"
        },
        "reasonings": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "thought": {
                        "type": "string",
                        "description": "A thought about the user's question"
                    },
                    "helpful": {
                        "type": "boolean",
                        "description": "Whether the thought is helpful to solving the user's question"
                    }
                }
            },
            "description": "Step-by-step reasoning process for query generation"
        },
        "generated_sql_query": {
            "type": "string",
            "description": "The final SQL query that answers the natural language question"
        }
    }
})

class Thought(BaseModel):
    """A thought about the user's question"""
    thought: str = Field(
        description="Text of the thought"
    )
    helpful: bool = Field(
        description="Whether the thought is helpful to solving the user's question"
    )

class FinalOutput(BaseModel):
    """Complete output structure containing the query, reasoning, and SQL"""
    user_nlp_query: str = Field(
        description="The original natural language query to be translated into SQL"
    )
    reasonings: List[Thought] = Field(
        description="Step-by-step reasoning process for query generation"
    )
    generated_sql_query: str = Field(
        description="The final SQL query that answers the natural language question"
    )

def make_prompt(question: str, schema_dict: Dict[str, Any], complexity: str) -> str:
    """
    Create prompt with appropriate instructions based on complexity
    """
    example_output = {
        "user_nlp_query": question,
        "reasonings": [
            {
                "thought": "First, we need to identify the main tables required",
                "helpful": True
            },
            {
                "thought": "Next, determine if any joins or aggregations are needed",
                "helpful": True
            },
            {
                "thought": "Finally, consider how to structure the WHERE clause",
                "helpful": True
            }
        ],
        "generated_sql_query": "SELECT column FROM table WHERE condition;"
    }

    base_prompt = f"""
You are an expert SQL developer with deep knowledge of database querying.
Your task is to generate a SQL query with clear reasoning steps.

QUESTION: {question}

SCHEMA INFORMATION:
{schema_dict}

THOUGHT INSTRUCTIONS
{thought_instructions}

REASONING INSTRUCTIONS
{reasoning_instructions}

REQUIRED OUTPUT FORMAT:
The response must be a valid JSON object exactly matching this schema:
{final_output_schema_json}

Example of properly formatted response:
{json.dumps(example_output, indent=2)}

REASONING REQUIREMENTS:
1. Provide 3-5 thoughts explaining your strategy
2. Each thought should explain WHY you're taking an approach
3. Focus on query planning, not implementation details
4. Consider table relationships and data types

QUERY COMPLEXITY LEVEL: {complexity}
"""

    if complexity == "EASY":
        base_prompt += "\nRESTRICTIONS: No JOINs or nested queries allowed."
    elif complexity == "NON-NESTED":
        base_prompt += "\nRESTRICTIONS: JOINs allowed but no nested queries."
    else:
        base_prompt += "\nRESTRICTIONS: Both JOINs and nested queries allowed if needed."
        
    base_prompt += "\n\nIMPORTANT: Return only valid JSON with no additional text."
    
    return base_prompt

def generate_gpt_response(prompt, max_retries=3):
    """
    Generate response from OpenAI API with retries
    """
    system_prompt = """You are an expert SQL developer. Always return responses as valid JSON matching the specified schema. Include detailed reasoning steps before generating SQL queries."""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o", 
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                n=1,
                stream=False,
                temperature=0.0,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0,
                response_format={"type": "json_object"}
            )
            content = response.choices[0].message.content
            print(f"Raw GPT response:\n{content}")  
            return content
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(3)
            else:
                print("Max retries reached")
                return None
    return None

def validate_gpt_response(response: str) -> bool:
    """
    Validate that GPT response contains all required fields
    """
    try:
        data = json.loads(response)
        required_fields = ["user_nlp_query", "reasonings", "generated_sql_query"]
        
        # Check all required fields exist in the output
        if not all(field in data for field in required_fields):
            print("Missing required fields in response")
            return False
            
        if not isinstance(data["reasonings"], list) or len(data["reasonings"]) < 1:
            print("Invalid reasonings array")
            return False
            
        # Check that each reasoning has required fields
        for reason in data["reasonings"]:
            if not all(field in reason for field in ["thought", "helpful"]):
                print("Invalid reasoning format")
                return False
                
        return True
    except Exception as e:
        print(f"Validation error: {str(e)}")
        return False

def process_question_sql(
    question: str,
    predicted_class: str,
    schema_dict: Dict[str, Any],
    max_retries: int = 3
) -> FinalOutput:
    """Generate SQL with thoughts and reasoning"""
    
    for attempt in range(max_retries):
        try:
            # Create appropriate prompt depending on classification module
            prompt = make_prompt(
                question=question,
                schema_dict=schema_dict,
                complexity=predicted_class
            )
            
            # Get GPT response
            response = generate_gpt_response(prompt)
            if response is None:
                continue
                
            # Validate format
            if not validate_gpt_response(response):
                print(f"Invalid response format on attempt {attempt + 1}")
                continue
                
            try:
                result = json.loads(response)
                return FinalOutput(
                    user_nlp_query=result["user_nlp_query"],
                    reasonings=[
                        Thought(**thought) for thought in result["reasonings"]
                    ],
                    generated_sql_query=result["generated_sql_query"]
                )
            except Exception as e:
                print(f"Error parsing response: {str(e)}")
                if attempt == max_retries - 1:
                    return FinalOutput(
                        user_nlp_query=question,
                        reasonings=[
                            Thought(
                                thought=f"Failed to parse response: {str(e)}",
                                helpful=False
                            )
                        ],
                        generated_sql_query="SELECT 1"
                    )
                continue
                
        except Exception as e:
            print(f"Process error: {str(e)}")
            if attempt == max_retries - 1:
                return FinalOutput(
                    user_nlp_query=question,
                    reasonings=[
                        Thought(
                            thought=f"Error in process: {str(e)}",
                            helpful=False
                        )
                    ],
                    generated_sql_query="SELECT 1"
                )
            continue
    
    return FinalOutput(
        user_nlp_query=question,
        reasonings=[
            Thought(
                thought="Maximum retries exceeded",
                helpful=False
            )
        ],
        generated_sql_query="SELECT 1"
    )

In [12]:
############################################ SELF CORRECTION
def debuger(question,sql, predicted_class, schema_dict):
	if '"EASY"' in predicted_class:
		prompt_used = easy_prompt_maker(
                    question=question,
                    schema_dict=schema_dict
                )
	elif '"NON-NESTED"' in predicted_class:
		prompt_used = medium_prompt_maker(
                    question=question,
                    schema_dict=schema_dict
                )
	else:
		prompt_used = hard_prompt_maker(
                    question=question,
                    schema_dict=schema_dict
                )

	instruction = """#### For the given question, use the provided tables, columns, foreign keys, and primary keys to check if the given SQLite SQL QUERY has any issues. If there are any issues, fix them and return the fixed SQLite QUERY in the output. If there are no issues, return the SQLite SQL QUERY as is in the output."
#### Background Information:
Relevant Schema Links:
{schema_links}
Prompt Used to Generate the Candidate SQLite SQL Query:
'''
{prompt_used}
'''
#### Use the following instructions for fixing the SQL QUERY: 
1) Use the database values that are explicitly mentioned in the question.
2) Pay attention to the columns that are used for the JOIN by using the Foreign_keys.
3) Use DESC and DISTINCT only when needed.
4) Pay attention to the columns that are used for the GROUP BY statement.
5) Pay attention to the columns that are used for the SELECT statement.
6) Only change the GROUP BY clause when necessary (Avoid redundant columns in GROUP BY).
7) Use GROUP BY on one column only."""
	prompt = instruction.format(
       schema_links=schema_dict["schema_links"],
	   prompt_used=prompt_used
   ) + f"""
#### Question: {question}
#### SQLite SQL QUERY
{sql}
#### SQLite FIXED SQL QUERY
"""
	return prompt



def GPT4_debug(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        stream = False,
        temperature=0.0,
        #max_tokens=350,
        top_p = 1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop = ["#", ";","\n\n"]
    )
    return response.choices[0].message.content


def refine_query(question, sql, classification, schema_dict):
	debugged_SQL = None
	while debugged_SQL is None:
		try:
			debugged_SQL = GPT4_debug(debuger(question,sql, classification, schema_dict))
		except:
			time.sleep(3)
			pass
	try:
		return debugged_SQL.split('```sql', 1)[1]
	except:
		raise IndexError
		

In [13]:
import sqlite3
import io
import csv
def execute_sql(query: str) -> str:
    conn = None
    try:
        conn = sqlite3.connect('sqlite/nport.db')
        cursor = conn.cursor()

        # Execute the query with a timeout
        cursor.execute(query)

        # Fetch column names and rows
        columns = [description[0] for description in cursor.description]
        rows = cursor.fetchall()

        # Convert the results to CSV
        output = io.StringIO()
        writer = csv.writer(output)
        writer.writerow(columns)
        writer.writerows(rows)
        csv_data = output.getvalue()
        output.close()

        return csv_data
    except sqlite3.Error as e:
        print(f"Database error: {str(e)}")
        raise e
    except Exception as e:
        print(f"Error executing SQL: {str(e)}")
        raise e
    finally:
        if conn:
            conn.close()

In [14]:
def compare_csv_strings(csv_data1: str, csv_data2: str) -> bool:
    # Use io.StringIO to read the CSV strings as file-like objects
    csv_file1 = io.StringIO(csv_data1)
    csv_file2 = io.StringIO(csv_data2)
    
    # Create CSV readers for each CSV string
    reader1 = csv.reader(csv_file1)
    reader2 = csv.reader(csv_file2)
    
    # Compare rows one by one
    for row1, row2 in zip(reader1, reader2):
        if row1 != row2:
            return False  # Rows are different
    
    # Check if there are extra rows in either file
    try:
        next(reader1)
        return False  # Extra rows in csv_data1
    except StopIteration:
        pass

    try:
        next(reader2)
        return False  # Extra rows in csv_data2
    except StopIteration:
        pass

    return True  # CSVs are identical


def compare_csv_din(ground_truth_query: str, llm_query: str, qnum):
    ## let LLM stack query the database
    append_to_file(f"Ground Truth Query: {ground_truth_query}", qnum)

    try: 
        schema_dict = global_vr.process_schema(llm_query)
        append_to_file(f"Schema Links for Question: {llm_query}\n{schema_dict}", qnum)
    except Exception as e:
        err_string = (f"Error in process_schema of Value Retrieval: {e}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    try:
        classification, class_reasoning = process_question_classification(llm_query, schema_dict)
        append_to_file(f"classification reasoning: {class_reasoning}", qnum)
        append_to_file(f"classification: {classification}", qnum)
    except Exception as e:
        err_string = (f"Error in process_question_classification of Classification: {e}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    try:
        process_thesql = process_question_sql(llm_query, classification, schema_dict)
        append_to_file(f"Thoughts: {process_thesql.reasonings}", qnum)
        append_to_file(f"SQL: {process_thesql.generated_sql_query}", qnum)
    except Exception as e:
        err_string = (f"Error in process_question_sql of SQL Generation: {e}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    try:
        final_output = refine_query(llm_query, process_thesql.generated_sql_query, classification, schema_dict)
        append_to_file(f"final_output: {final_output}", qnum)
    except Exception as e:
        err_string = (f"Error in refine_query of Self-Correction: {e}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    try:
        llm_csv = execute_sql(final_output.replace("```sql", "").replace("```", "").strip())
    except Exception as e:
        err_string = (f"Error Executing LLM-Generated SQL: {str(e)}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    
    
    try:
    ## direct query to the database
        ground_truth_csv = execute_sql(ground_truth_query)
    except Exception as e:
        err_string = (f"Error Executing Ground Truth SQL: {str(e)}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e
    ## compare results
    
    try:
        diff=compare_csv_strings(ground_truth_csv,llm_csv)
        if diff:
            print("CSV outputs match perfectly.")
            return True
        else:
            print("Mismatch found.")
            return False
    except Exception as e:
        err_string=(f"Error comparing CSVs: {str(e)}")
        print(err_string)
        append_to_file(err_string,qnum)
        raise e


In [15]:
import csv
import json

# Load data from query_summary.csv
def load_queries(input_file):
    llm_query = []
    ground_truth_query = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            llm_query.append(row["Question"])
            ground_truth_query.append(row["SQL"])
    return llm_query, ground_truth_query

# Save arrays to file
def save_queries_to_file(file_path, llm_query, ground_truth_query):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump({"llm_query": llm_query, "ground_truth_query": ground_truth_query}, file)

# Load arrays from file
def load_queries_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data["llm_query"], data["ground_truth_query"]

# Write comparison results to output file
def write_to_output(file_path, ground_truth, llm, result):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([ground_truth, llm, result])

# Main script logic
def process_queries(input_file, temp_file, output_file):
    # If temp file doesn't exist, load queries from CSV
    try:
        llm_query, ground_truth_query = load_queries_from_file(temp_file)
    except FileNotFoundError:
        llm_query, ground_truth_query = load_queries(input_file)
        save_queries_to_file(temp_file, llm_query, ground_truth_query)

    # Prepare output file with a header if starting fresh
    if not os.path.exists(output_file):
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Ground_Truth_Query", "LLM_Query", "Comparison_Result"])
    i = 0
    # Process queries
    while llm_query and ground_truth_query:
        gt_query = ground_truth_query.pop(0)
        llm_query_item = llm_query.pop(0)

        # Compare queries and write results
        try:
            print("========================================================================================================")
            print("========================================================================================================")
            print("========================================================================================================")
            print(f"Currently processing Question {i}")
            print("========================================================================================================")
            print("========================================================================================================")
            print("========================================================================================================")
            result = compare_csv_din(gt_query, llm_query_item, i)
            append_to_file(f"Result: {result}", i)
            write_to_output(output_file, gt_query, llm_query_item, result)
            
        except Exception as e:
            write_to_output(output_file, gt_query, llm_query_item, "Error")

        # Save the remaining queries back to the temp file
        save_queries_to_file(temp_file, llm_query, ground_truth_query)
        i+=1


In [16]:
print(os.getcwd())

/Users/hannahzhang/Desktop/PIMCO 1B/PIMCO-Text2SQL


In [17]:
input_file = "query_summary.csv"
temp_file = "temp_queries.json"  # Temporary file to store remaining queries
output_file = "din_accuracy_120.csv"

process_queries(input_file, temp_file, output_file)


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "Given a financial database schema: {'type': 'database', 'schema': {'tables': [{'name': 'REGISTRANT', 'columns': [{'name': 'ACCESSION_NUMBER', 'type': 'TEXT'}, {'name': 'CIK', 'type': 'TEXT'}, {'name': 'REGISTRANT_NAME', 'type': 'TEXT'}, {'name': 'FILE_NUM', 'type': 'TEXT'}, {'name': 'LEI', 'type': 'TEXT'}, {'name': 'ADDRESS1', 'type': 'TEXT'}, {'name': 'ADDRESS2', 'type': 'TEXT'}, {'name': 'CITY', 'type': 'TEXT'}, {'name': 'STATE', 'type': 'TEXT'}, {'name': 'COUNTRY', 'type': 'TEXT'}, {'name': 'ZIP', 'type': 'TEXT'}, {'name': 'PHONE', 'type': 'TEXT'}]}, {'name': 'FUND_REPORTED_INFO', 'columns': [{'name': 'ACCESSION_NUMBER', 'type': 'TEXT'}, {'name': 'SERIES_NAME', 'type': 'TEXT'}, {'name': 'SERIES_ID', 'type': 'TEXT'}, {'name': 'SERIES_LEI', 'type': 'TEXT'}, {'name': 'TOTAL_ASSETS', 'type': 'TEXT'}, {'name': 'TOTAL_LIABILITIE

Currently processing Question 0


DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x12db74550>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 03 Dec 2024 11:25:15 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'1225'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining

Found 254 matches for 'fund':
  securities_lending.is_loan_by_fund: 1.0000
  registrant.accession_number: 0.9000
  registrant.cik: 0.9000
  registrant.registrant_name: 0.9000
  registrant.file_num: 0.9000
Found 254 matches for 'total':
  fund_reported_info.total_assets: 1.0000
  fund_reported_info.total_liabilities: 1.0000
  monthly_total_return.monthly_total_return_id: 1.0000
  monthly_total_return.monthly_total_return1: 1.0000
  monthly_total_return.monthly_total_return2: 1.0000
Found 5 matches for 'largest':
  submission.is_last_filing: 0.7273
  floating_rate_reset_tenor.rate_reset_tenor_id: 0.6667
  floating_rate_reset_tenor.reset_date: 0.6667
  floating_rate_reset_tenor.reset_date_unit: 0.6667
  swaption_option_warnt_deriv.shares_cnt: 0.6154
Found 1 matches for 'top':
  fund_reported_info.assets_attrbt_to_misc_security: 0.8000
Found 7 matches for 'asset':
  fund_reported_holding.asset_cat: 1.0000
  monthly_return_cat_instrument.asset_cat: 1.0000
  fund_reported_info.total_assets: 

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 03 Dec 2024 11:25:20 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'2409'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1986150'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'415ms'), (b'x-request-id', b'req_1b34957c8ae143bcd70447ec06326661'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=u61.Sri_1bCs6S22gjJSt1yqUnr1vmF2763IcnR9zjc-1733225120-1.0.1.1-_jht754QG0tQfgJUrnuRrM2g4YKkFRTtaZntY3sss

...

Found 254 matches for 'fund':
  securities_lending.is_loan_by_fund: 1.0000
  registrant.accession_number: 0.9000
  registrant.cik: 0.9000
  registrant.registrant_name: 0.9000
  registrant.file_num: 0.9000
Found 29 matches for 'ratio':
  convertible_security_currency.conversion_ratio: 1.0000
  interest_rate_risk.interest_rate_risk_id: 0.9000
  interest_rate_risk.intrst_rate_change_3mon_dv01: 0.9000
  interest_rate_risk.intrst_rate_change_1yr_dv01: 0.9000
  interest_rate_risk.intrst_rate_change_5yr_dv01: 0.9000
Found 7 matches for 'asset':
  fund_reported_holding.asset_cat: 1.0000
  monthly_return_cat_instrument.asset_cat: 1.0000
  fund_reported_info.total_assets: 0.9000
  fund_reported_info.net_assets: 0.9000
  fund_reported_info.assets_attrbt_to_misc_security: 0.9000
Found 2 matches for 'liability':
  fund_reported_info.total_liabilities: 0.9000
  interest_rate_risk.interest_rate_risk_id: 0.6545
Found 5 matches for 'highest':
  fund_reported_info.credit_spread_3mon_invest: 0.6154
  fun

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 03 Dec 2024 11:27:05 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'3299'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1986137'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'415ms'), (b'x-request-id', b'req_10a964fff76f97f81cf3bb1a4f677e6e'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8ec334871c742aa7-LAX'), (b'Content-Encoding', b

Raw response: To determine which funds have the highest liabilities to assets ratio, we need to calculate this ratio for each fund and then identify the funds with the highest values. This involves using data from the "FUND_REPORTED_INFO" table, specifically the "TOTAL_LIABILITIES" and "TOTAL_ASSETS" columns. 

The process involves:
1. Calculating the liabilities to assets ratio for each fund.
2. Sorting the results to find the highest ratios.

Since this requires operations on a single table without the need for joining with other tables or using nested queries, the SQL query can be classified as "EASY."
Trying to extract classification from: To determine which funds have the highest liabilities to assets ratio, we need to calculate this ratio for each fund and then identify the funds with the highest values. This involves using data from the "FUND_REPORTED_INFO" table, specifically the "TOTAL_LIABILITIES" and "TOTAL_ASSETS" columns. 

The process involves:
1. Calculating the liabilit

DEBUG:httpcore.http11:receive_response_headers.failed exception=KeyboardInterrupt()
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete


KeyboardInterrupt: 

In [ ]:
input_file = "query_summary.csv"
temp_file = "temp_queries.json"  # Temporary file to store remaining queries
output_file = "din_accuracy_120.csv"

process_queries(input_file, temp_file, output_file)
